[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/enterprise/healthcare/NER_Spanish.ipynb)

In [ ]:
import json

with open('keys.json') as f:
    license_keys = json.load(f)

license_keys.keys()


dict_keys(['SPARK_VERSION', 'SPARK_NLP_PUBLIC_VERSION', 'SPARK_NLP_VERSION', 'SPARK_NLP_SECRET', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'JSL_OCR_LICENSE', 'JSL_OCR_SECRET'])

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

secret = license_keys.get("secret",license_keys.get('SPARK_NLP_SECRET', ""))
spark_version = os.environ.get("SPARK_VERSION", license_keys.get("SPARK_VERSION","2.4"))
version = license_keys.get("version",license_keys.get('SPARK_NLP_PUBLIC_VERSION', ""))
jsl_version = license_keys.get("jsl_version",license_keys.get('SPARK_NLP_VERSION', ""))

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['JSL_OCR_LICENSE'] = license_keys['JSL_OCR_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']

print(spark_version, version, jsl_version)

! python -m pip install "pyspark==$spark_version".*
! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

print (sparknlp.version())
print (sparknlp_jsl.version())

spark = sparknlp_jsl.start(secret, gpu=False, spark23=(spark_version[:3]=="2.3"))

openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
2.3.3 2.5.5 2.5.5
     |████████████████████████████████| 211.9MB 58kB/s 
     |████████████████████████████████| 204kB 46.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.3.3-py2.py3-none-any.whl size=212321782 sha256=270a09d2572184d92a2dae3648ecdf1c323554a244f2cc79ec42a647da298ad3
  Stored in directory: /root/.cache/pip/wheels/46/c0/d7/d38597c9bf05be6a99f6c5fcc200dabccfb2bb3af9a30f3f76
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/2.5.5-4f4b7f600f8ba3cdc5973a6baa47b901b0c8d8a3
     |████████████████████████████████| 133kB 2.8MB/s 
2.5.5
2.5.5


In [ ]:
data_chunk_merge = spark.createDataFrame([
  (1,"""ANAMNESIS
Mujer de 67 años con antecedentes personales de hipotiroidismo en tratamiento con levotiroxina y fumadora activa de 12.5 paquetes/año. Consulta en Urgencias por sensación progresiva de “acorchamiento y hormigueos” en ambos miembros superiores e inferiores, así como una dificultad progresiva para la deambulación de 4 meses de evolución, asociando asimismo alteración de la memoria reciente desde el último mes
EXPLORACIÓN FÍSICA
Presentaba una exploración cardiopulmonar y abdominal normal. En la exploración neurológica destaca balance motor por grupos musculares conservado; arreflexia rotuliana y aquilea e hiporreflexia bicipital; tetrahipoestesia asimétrica (táctil, algésica, vibratoria y posicional) en patrón de “guante y calcetín altos” de predominio izquierdo; Romberg positivo y marcha con leve aumento de base de sustentación que impresiona ataxia sensitiva.
PRUEBAS COMPLEMENTARIAS
En las exploraciones complementarias, hemograma, bioquímica y coagulación no presentaban alteraciones. El análisis de virus hepatitis B y C, VIH, las serologías de enfermedad de Lyme y Treponema pallidum fueron negativas. El estudio lipídico, de
vitaminas, proteinograma y anticuerpos antinucleares fueron normales. Se objetivó positividad para los anticuerpos onconeuronales anfifisina, anti-Hu y anti-SOX-1.
Se realizó una punción lumbar. El estudio bacteriológico, de micobacterias, de Herpes virus 1 y 2, Ebstein Barr, Citomegalovirus y Varicela-Zoster fue negativo.
El estudio de electroneurografía-electromiografía (ENG-EMG) demostró ausencia de potenciales sensitivos en miembros inferiores y algunos nervios de miembros superiores; siendo las respuestas presentes de amplitud muy reducida, con marcado alargamiento de latencias y disminución severa de la velocidad de conducción. Estudio de conductividad motora y electromiográfico sin hallazgos patológicos, hallazgos compatibles con polineuropatía sensitiva severa de características mixtas.
En la tomografía axial computerizada (TAC) cerebral no se apreciaron alteraciones significativas. En la TAC toraco-abdomino-pélvica se observaron una adenopatía parahiliar derecha de 2.5 cm y una adenopatía subcarinal de características patológicas.
En el PET-TAC se observaron depósitos patológicos que coincidían con los hallazgos del TAC.
Se realizó una ecobroncoscopia lineal con toma de biopsia de adenopatía subcarinal.
La resonancia mágnetica (RM) cervico-dorso-lumbar mostró pequeñas hernias discales dorsales y lumbares sin afectación del canal medular ni de la cola de caballo.
La RM cerebral mostró en T2 hiperintensidad en ambos lóbulos temporales alcanzando hipocampo y corteza siendo sugestivo de encefalitis límbica.
ANATOMÍA PATOLÓGICA
Carcinoma microcítico de pulmón. Inmunohistoquímica: positivo para TTF1, cromogranina y sinaptofisina, negativo para CK7, CK20 y p40.
JUICIO DIAGNÓSTICO
Encefalitis límbica y polineuropatía sensitiva paraneoplásicas secundarias a carcinoma microcítico de pulmón cTxN2 M0 (enfermedad limitada).
TRATAMIENTO Y EVOLUCIÓN
Ante el juicio diagnóstico, y con la colaboración del Servicio de Neurología, se inició tratamiento sintomático secuencial con gammaglobulina (0.4 mg/kg/día) y corticoides (metilprednisolona 1 g/día) intravenosos durante 5 días respectivamente, con escasa mejoría de la clínica. Debido a las molestias ocasionadas por las parestesias se inició tratamiento oral con pregabalina a dosis de 50-0-75 mg.
Se decidió comenzar con tratamiento con cisplatino-etopósido más radioterapia concomitante (59.4 Gy en 30 sesiones) a partir del segundo ciclo. Se completó un total de cinco ciclos observándose una respuesta parcial radiológica por criterios RECIST (Response Evaluation Criteria In Solid Tumors).
Tres meses después de la finalización del tratamiento con quimioterapia la paciente refiere empeoramiento progresivo de la clínica polineuropática, presentando también episodios súbitos de fuertes reacciones de miedo, llanto y ansiedad, congruentes con crisis epilépticas focales límbicas, por lo que se decide administrar nueva tanda de gammaglobulinas iv (0,4 mg/kg/día) e iniciar tratamiento antiepiléptico con levetirazetam 500 mg cada 12h y lacosamida 100 mg cada 12h, consiguiendo mejoría sintomática cognitiva y control de las crisis. Posteriormente ante mayor progresión de la clínica polineuropática se decide tratamiento de 2ª línea con rituximab iv (1000 mg separados entre sí 14 días), manteniéndose clínicamente estable.
Después de nueve meses libre de progresión, se objetiva en el TAC de control un aumento de tejido de partes blandas perihiliar derecho y un nódulo de 7 mm en lóbulo inferior derecho no presente en el TAC previo, hipermetabólico en el PET-TC. Por lo tanto, ante una progresión de enfermedad se decide reiniciar quimioterapia con carboplatinoetopósido al ser una paciente platino sensible."""),
  (2,"""Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias. Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado. Entre los comensales aparecieron varios casos de brucelosis. Durante el ingreso para estudio del síndrome febril con antecedentes epidemiológicos de posible exposición a Brucella presenta un cuadro de orquiepididimitis derecha.
La exploración física revela: Tª 40,2 C; T.A: 109/68 mmHg; Fc: 105 lpm. Se encuentra consciente, orientado, sudoroso, eupneico, con buen estado de nutrición e hidratación. En cabeza y cuello no se palpan adenopatías, ni bocio ni ingurgitación de vena yugular, con pulsos carotídeos simétricos. Auscultación cardíaca rítmica, sin soplos, roces ni extratonos. Auscultación pulmonar con conservación del murmullo vesicular. Abdomen blando, depresible, sin masas ni megalias. En la exploración neurológica no se detectan signos meníngeos ni datos de focalidad. Extremidades sin varices ni edemas. Pulsos periféricos presentes y simétricos. En la exploración urológica se aprecia el teste derecho aumentado de tamaño, no adherido a piel, con zonas de fluctuación e intensamente doloroso a la palpación, con pérdida del límite epidídimo-testicular y transiluminación positiva.
Los datos analíticos muestran los siguentes resultados: Hemograma: Hb 13,7 g/dl; leucocitos 14.610/mm3 (neutrófilos 77%); plaquetas 206.000/ mm3. VSG: 40 mm 1ª hora. Coagulación: TQ 87%; TTPA 25,8 seg. Bioquímica: Glucosa 117 mg/dl; urea 29 mg/dl; creatinina 0,9 mg/dl; sodio 136 mEq/l; potasio 3,6 mEq/l; GOT 11 U/l; GPT 24 U/l; GGT 34 U/l; fosfatasa alcalina 136 U/l; calcio 8,3 mg/dl. Orina: sedimento normal.
Durante el ingreso se solicitan Hemocultivos: positivo para Brucella y Serologías específicas para Brucella: Rosa de Bengala +++; Test de Coombs > 1/1280; Brucellacapt > 1/5120. Las pruebas de imagen solicitadas ( Rx tórax, Ecografía abdominal, TAC craneal, Ecocardiograma transtorácico) no evidencian patología significativa, excepto la Ecografía testicular, que muestra engrosamiento de la bolsa escrotal con pequeña cantidad de líquido con septos y testículo aumentado de tamaño con pequeñas zonas hipoecoicas en su interior que pueden representar microabscesos.
Con el diagnóstico de orquiepididimitis secundaria a Brucella se instaura tratamiento sintomático (antitérmicos, antiinflamatorios, reposo y elevación testicular) así como tratamiento antibiótico específico: Doxiciclina 100 mg vía oral cada 12 horas (durante 6 semanas) y Estreptomicina 1 gramo intramuscular cada 24 horas (durante 3 semanas). El paciente mejora significativamente de su cuadro tras una semana de ingreso, decidiéndose el alta a su domicilio donde completó la pauta de tratamiento antibiótico. En revisiones sucesivas en consultas se constató la completa remisión del cuadro.""")
]).toDF("id","text")

In [ ]:
da = DocumentAssembler().setInputCol("text").setOutputCol("document")
sd = SentenceDetector().setInputCols("document").setOutputCol("sentence")
tk = Tokenizer().setInputCols("sentence").setOutputCol("token")
emb = WordEmbeddingsModel.pretrained("embeddings_scielowiki_300d","es","clinical/models").setOutputCol("embs")
ner = NerDLModel.pretrained("ner_diag_proc","es","clinical/models").setInputCols("sentence","token","embs").setOutputCol("ner")
canner = NerDLModel.pretrained("ner_neoplasms","es","clinical/models").setInputCols("sentence","token","embs").setOutputCol("canner")
nc = NerConverter().setInputCols("sentence","token","ner").setOutputCol("ner_chunk")
cannc = NerConverter().setInputCols("sentence","token","canner").setOutputCol("canner_chunk")

embeddings_scielowiki_300d download started this may take some time.
Approximate size to download 351.2 MB
[OK!]
ner_diag_proc download started this may take some time.
Approximate size to download 14.2 MB
[OK!]
ner_neoplasms download started this may take some time.
Approximate size to download 14.1 MB
[OK!]


In [ ]:
#######################################################################################

merger_can = ChunkMergeApproach().setInputCols("ner_chunk","canner_chunk").setOutputCol("combined")

#######################################################################################

pl = Pipeline().setStages([da,sd,tk,emb,ner,canner,nc,cannc,merger_can])

In [ ]:
merged_data = pl.fit(data_chunk_merge).transform(data_chunk_merge).cache()

In [ ]:
merged_data.selectExpr("id","explode(ner_chunk) as a")\
.selectExpr("id","a.begin","a.end","a.result","a.metadata.entity")\
.orderBy("id","begin").show(100, False)

+---+-----+----+----------------------------------------+-------------+
|id |begin|end |result                                  |entity       |
+---+-----+----+----------------------------------------+-------------+
|1  |58   |71  |hipotiroidismo                          |DIAGNOSTICO  |
|1  |107  |114 |fumadora                                |DIAGNOSTICO  |
|1  |367  |390 |alteración de la memoria                |DIAGNOSTICO  |
|1  |588  |597 |arreflexia                              |DIAGNOSTICO  |
|1  |621  |633 |hiporreflexia                           |DIAGNOSTICO  |
|1  |864  |879 |ataxia sensitiva                        |DIAGNOSTICO  |
|1  |1030 |1040|hepatitis B                             |DIAGNOSTICO  |
|1  |1047 |1049|VIH                                     |DIAGNOSTICO  |
|1  |1070 |1079|enfermedad                              |DIAGNOSTICO  |
|1  |1084 |1087|Lyme                                    |DIAGNOSTICO  |
|1  |1091 |1099|Treponema                               |DIAGNOS

In [ ]:
merged_data.selectExpr("id","explode(canner_chunk) as a")\
.selectExpr("id","a.begin","a.end","a.result","a.metadata.entity")\
.orderBy("id","begin").show(100, False)

+---+-----+----+---------------------+--------------------+
|id |begin|end |result               |entity              |
+---+-----+----+---------------------+--------------------+
|1  |2710 |2730|Carcinoma microcítico|MORFOLOGIA_NEOPLASIA|
|1  |2940 |2960|carcinoma microcítico|MORFOLOGIA_NEOPLASIA|
|1  |2978 |2979|M0                   |MORFOLOGIA_NEOPLASIA|
+---+-----+----+---------------------+--------------------+



In [ ]:
merged_data.selectExpr("id","explode(combined) as a")\
.selectExpr("id","a.begin","a.end","a.result","a.metadata.entity")\.show(100, False)